In [12]:
import pandas as pd
import numpy as np
import ast
import seaborn
from collections import Counter
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
from bs4 import BeautifulSoup
import logging, gensim, bz2
import os
import re
from operator import itemgetter


import json
import urlparse
from itertools import chain
flatten = chain.from_iterable
from nltk import word_tokenize


def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()


montreal_review = load("montreal_review.pkl")


corpus = load("corpus_montreal.pkl")
#dictionary = load("dictionary_montreal.pkl")


#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, passes=20)
#print(ldamodel.print_topics(topics=50, topn=10))


#ldamodel.save("lda_montreal_1.model")


model =  gensim.models.LdaModel.load('lda_montreal_1.model')


model.print_topics(50,topn=30)




[u'0.087*egg + 0.086*brunch + 0.083*breakfast + 0.032*toast + 0.031*fruit + 0.024*bacon + 0.024*benedict + 0.022*morn + 0.019*potato + 0.019*french + 0.017*coffe + 0.017*juic + 0.016*pancak + 0.013*fresh + 0.013*ham + 0.011*omelett + 0.011*orang + 0.011*came + 0.011*sausag + 0.009*weekend + 0.009*menu + 0.009*side + 0.008*smoothi + 0.008*chees + 0.008*salmon + 0.008*waffl + 0.007*crepe + 0.007*plate + 0.006*spot + 0.006*serv',
 u'0.037*drink + 0.030*beer + 0.030*great + 0.028*place + 0.027*bar + 0.026*good + 0.024*night + 0.018*friend + 0.014*music + 0.013*food + 0.013*nice + 0.011*atmospher + 0.011*cocktail + 0.010*select + 0.010*fun + 0.008*realli + 0.007*like + 0.007*loud + 0.007*staff + 0.007*cool + 0.007*enjoy + 0.007*menu + 0.007*get + 0.007*dinner + 0.007*group + 0.006*also + 0.006*pretti + 0.006*date + 0.006*crowd + 0.006*bartend',
 u'0.241*burger + 0.112*steak + 0.058*fri + 0.031*frite + 0.027*bun + 0.024*mussel + 0.022*medium + 0.021*cook + 0.018*rare + 0.017*patti + 0.014*ha

In [13]:
term_list = []
n_topics = 50
for i in range(0, n_topics):
    temp = model.show_topic(i, 20)
    terms = []
    for term in temp:
            terms.append(term)
    term_list.append(terms)
    
topic_dict = []
for name in os.listdir("moteral_WordCloudLabels"):
    if name != ".DS_Store":

        topic = name
    
        for filename in os.listdir("moteral_WordCloudLabels/" + name):
            if filename != ".DS_Store":
                print(filename)
                topic_id = int(re.search(r'\d+', filename).group())
                sub_topic_re = re.search(r'_\w+', filename)
                if sub_topic_re:
                     sub_topic = sub_topic_re.group()[1:]
                else:
                    sub_topic=''
                topic_dict.append([topic_id, topic, sub_topic])
    
topic_dict = pd.DataFrame(topic_dict, columns=['Topic_ID', 'Topic', 'Sub_Topic'])


topic_dict = topic_dict.set_index('Topic_ID').to_dict()


topic_dict['Topic'][1]




Topic_list = []
for corp in corpus:
    _srtd = sorted(model[corp], key=itemgetter(1), reverse=1)
    top, score = _srtd[0]
    top_name = topic_dict['Topic'][top] 
    top_level2 = topic_dict['Sub_Topic'][top]
    all_ID = [tup[0] for tup in _srtd]
    all_name = [topic_dict['Topic'][id] for id in all_ID]
    all_level2 = [topic_dict['Sub_Topic'][id] for id in all_ID]
    top_topic_words = [tup[1] for tup in term_list[top]]
    Topic_list.append([top, top_name, top_level2, score, all_ID, all_name, all_level2, _srtd, top_topic_words])


Topic_list =  pd.DataFrame(Topic_list, columns=['Top_ID', 'Top_name', 'Top_level2', 
                                                'Score_top', 'All_ID', 'All_name',
                                                'All_level2', 'All_score', 'Top_Topic_Words'])


#montreal_review.head()



terms3_ambience.png
terms1_alchol_beer_bar.png
terms0_breakfast.png
terms10_spanish.png
terms11_deserts_icecream.png
terms13_asian.png
terms14_japanese.png
terms15_pizza.png
terms16_coffee.png
terms24_duck.png
terms25_chicken.png
terms26_irish.png
terms27_subs.png
terms28_mexican.png
terms2_burger_fries.png
terms31_taco.png
terms33_salad.png
terms39_thai.png
terms47_indian.png
terms4_vegetarian.png
terms5_meat.png
terms9_greek.png
terms22_perfect_food.png
terms35_food.png
terms44_food_good.png
terms36_place.png
terms17.png
terms18.png
terms19.png
terms20.png
terms21.png
terms29.png
terms30.png
terms37.png
terms38.png
terms41.png
terms42.png
terms43.png
terms45.png
terms46.png
terms48.png
terms49.png
terms7.png
terms8.png
terms34_price.png
terms12_pay.png
terms23_waiting_time_seat.png
terms32_seat.png
terms40_wait.png
terms6_time.png


In [14]:
reviews = montreal_review[['user_id', 'review_id', 'text', 'name', 'attributes', 'categories', 'stars_x', 'stars_y']].reset_index()
reviews_labeled = pd.concat([reviews, Topic_list], axis=1)
reviews_labeled.to_csv('montreal_reviews_with_topics.csv')
reviews_labeled.to_pickle('montreal_reviews_with_topics.pkl')